# Estimating Non-Linear Macroeconomic Models in Julia

**Michael Cai** (github: @caimichael)

Senior Research Analyst, Federal Reserve Bank of New York (github: @FRBNY-DSGE)

<sub>*Special thanks to Ed Herbst and Frank Schorfheide for providing slide content from their 2017 CEF Workshop on Sequential Monte Carlo Methods and the Tempered Particle Filter.

# Disclaimer
This talk reflects the experience of the author and does not represent an endorsement by the Federal Reserve Bank of New York or the Federal Reserve System of any particular product or service. The views expressed in this talk are those of the author and do not necessarily reflect the position of the Federal Reserve Bank of New York or the Federal Reserve System. Any errors or omissions are the responsibility of the author.

# Outline
1. State Space Models and StateSpaceRoutines.jl
2. The Tempered Particle Filter
3. Computational Details
4. Conclusion

## State Space Models and StateSpaceRoutines.jl

### Background

- A dynamic stochastic general equilibrium (DSGE) model is a macroeconomic model built on microeconomic foundations that is used to forecast macroeconomic variables and to conduct policy exercises.


- **DSGE models can be represented as state space models!**


- In previous years, we have presented talks on our main package, [DSGE.jl](https://github.com/FRBNY-DSGE/DSGE.jl), a codebase for the estimation and forecasting of DSGE models. The package also implements various kinds of DSGE models, including our very own New York Fed DSGE model.

(see [Pearl Li's talk](https://www.youtube.com/watch?v=NH9AgnbJmVA) on parallelized forecasting at JuliaCon2017 and [Erica Moszkowski's talk](https://www.youtube.com/watch?v=Vd2LJI3JLU0) on the DSGE.jl package at JuliaCon2016)

### StateSpaceRoutines.jl

**StateSpaceRoutines.jl** is a package containing a variety of filtering and smoothing methods for estimating state space models, including:
- The Kalman filter and tempered particle filter.
- Hamilton smoother, Durbin Koopman smoother, and others.

### What is a State Space Model?

A **state space model** is a way to track the dynamic behavior of a set of unobserved variables by using noisy measurements.

<img src="figures/rover.jpg" width="400">

### How is a State Space Model Represented?

A **general state space model** can be written as a system of two equations:

**Transition:** $$s_t = \Phi(\theta)(s_{t-1}, \epsilon_t) \text{ , } \epsilon_t \sim F_\epsilon(\theta)$$

**Measurement:** $$y_t = \Psi(\theta)(s_t) + u_t \text{ , } u_t \sim F_u(\theta)$$

### Example Variables from the New York Fed DSGE Model

| Notation     | Name        | Examples                                   |
| ----------------------------------------------------------------------- |
| $s_t$        | States      | Output growth, inflation                   |
| $y_t$        | Observables | Real GDP growth, core PCE inflation        |
| $\epsilon_t$ | Shocks      | Productivity shock, aggregate demand shock |
| $\theta$     | Parameters  | Household discount rate, inflation target  |

### Why non-linear models?

- Most linear models are linear approximations of a non-linear model.
- Sometimes, the non-linearities in a model are *fundamental* to explaining certain phenomena.

### Examples of non-linear models

**Economics**:
- Models that incorporate the zero lower bound on the nominal interest rate

**Finance**:
- Stochastic volatility models

**Fluid Dynamics**:
- Turbulent flow models

**Environmental Science**:
- Ecological prediction models

### Particle Filters

Insert layout (2,1) plot with normal distr. on the left hand side, and a bi-modal histogram on the right hand side.

# The Tempered Particle Filter

Introduced in *"Tempered Particle Filtering"* by Ed Herbst (Federal Reserve Board of Governors) and Frank Schorfheide (University of Pennsylvania), to be published in an upcoming edition of the Journal of Econometrics.

Working paper available [here](https://economics.yale.edu/sites/default/files/temperedparticlefilter_v15-wivoln.pd_.pdf)

### The Algorithm

1. **Initialization**: Draw $N$ particles $s_0 \sim p(s_0)$ and initialize $N$ weights $W_0$ each to be $1$.
2. **Recursion**: for $t$ in $1,...,T$
    1. **Forecast** $s_t$ with the transition equation.
    2. **Correction**: Calculate the weights $W_t$ of each particle.
    3. **Selection**: Resample the particles, $s_t$.
    4. **Sequential Monte Carlo Sampler**
        1. Correction
        2. Selection
        3. Mutation

### The Genetic Interpretation

<img src="figures/correction_selection_mutation.png" width="650">

**Figure 1**: An Illustration of Correction-Selection-Mutation

### What determines the fitness of the herd?

<img src="figures/weighting_cropped.png" width="500"> <br>
**Figure 2**: Understanding Importance Sampling

*Computing in Economics and Finance Workshop 2017 by Ed Herbst and Frank Schorfheide*

### The Weaknesses of the Bootstrap Particle Filter

- The lack of diversity in particle values, leading to less accurate Monte Carlo approximations.
- The inability to adapt to periods in which highly unlikely data is realized.

### The Key Insight of the Tempered Particle Filter

Increasing the measurement error variance, $\Sigma_u$, holding all else constant, *decreases* the variance of the particle weights.


If the measurement errors are $N(0, \Sigma_u)$ then the weights are of the form:

$$ W^j_t \propto exp\big\{-\frac{1}{2}(y_t - \Psi(\tilde{s}^j_t))^\prime \Sigma^{-1}_u(y_t - \Psi(\tilde{s}^j_t))\big\}$$

### Using Inflated Measurement Error to build Bridge Distributions

<img src="figures/tempering_iterations.png" width="700"> <br>

**Figure 3**: Bridge Distributions, $p_n(s_t|y_t)$

*Computing in Economics and Finance Workshop 2017 by Ed Herbst and Frank Schorfheide*

### Benchmark against the Kalman Filter

This figure shows the predictive density, $p(y_t|y_{1:t-1})$ for $t = 1, ..., T$ from the linear state space representation of the basic 3-equation New Keynesian model. The bootstrap and tempered particle filter were both run 10 times to produce these results.

<img src="figures/bootstrap_v_tpf.png" width="600"> 

In [1]:
# Measuring the filtered states (choose n = 8000)

# Grab the same modal parameter that was used to generate the system for the above figure.
# Calculate Ψ(s_filt) and plot against the true data for GDP and CPI (horizontal subplots)

# Computational Details

### Parallel Computing in Macroeconomics

As macroeconomic models become increasingly complex, the methods used to estimate them have become increasingly sophisticated.

One common characteristic of many of these methods is that they are parallelizable on some level.

### Parallel Computing Considerations

- The size of the computation being parallelized over, and the overall size of the algorithm.
- The relevant share of time spent conducting network I/O versus actually computing.

### Performance of TPF Variants ($n_{particles} = 8000$)

Add distributed parallel implementation of tpf to this figure

<img src="figures/parallel_v_serial_time.png" width="650"> <br>

### Computation Hurdles We're Still Facing

Fortran is still faster by 1 order of magnitude.

Insert time table here.

# Conclusion

### Ongoing work
- The tempered particle filter and StateSpaceRoutines.jl
- Sequential Monte Carlo sampling in DSGE.jl - a parallelized and robust alternative to Random Walk Metropolis Hastings sampling.
- Heterogeneous Agent DSGE models in DSGE.jl

### Acknowledgements
- NY Fed DSGE Team: Marco Del Negro, Ethan Matlin, and Reca Sarfati
- NY Fed DSGE Alumni: Pearl Li and Abhi Gupta (good luck in graduate school!)

### Thank you!